# HW5 (Last homework): designing a microscope using machine learning

For the final homework assignment, we will design a microscope using machine learning! In order to do this, we will combine elements of HW3 (CNNs) and HW4 (microscope simulator) into a single end-to-end architecture. In particular, we will convert the MNIST digits into phase/amplitude objects, and then process them through a microscope simulator as we did in HW4, add noise, and then feed the resulting images into a CNN for classification. In addition to optimizing the CNN parameters, we will also be simultaneously optimizing the input illumination incident on the phase/amplitude objects, as well as the shape of the microscope's aperture, to try to improve classification performance.

Below, we will walk you through the steps of implementing this joint architecture, leaving some portions blank for you to implement. We will first instruct you to use specific values that we have tested and are known to give reasonable results. Later on, you will revisit the code and explore different hyperparameter settings.

While the code below is written in low-level eager mode, if you would prefer to rewrite it using high-level keras with a custom layer, feel free.

Some code adapted from https://www.tensorflow.org/tutorials/quickstart/advanced


In [0]:
# if running in colab, this block chooses tf 2.0:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

## 1. import stuff and load MNIST
As always, we split the dataset into training and testing.

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
assert '2.' in tf.__version__  # make sure you're using tf 2.0

# load mnist dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # customary normalization to [0, 1]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

## 2. convert the MNIST images into microscope samples
Convert the MNIST images into phase-only objects. To do this, normalize the MNIST digits to be between 0 and 1 (which was done above), and make the object 1 wavelength thick (we may come back later to adjust the sample thickness). We'll assume the wavelength of the light in this microscope is 500 nm, or 0.5 µm.

In [0]:
wavelength = .5
def convert_MNIST(X):
    # your code here
    pass

x_train = convert_MNIST(x_train)
x_test = convert_MNIST(x_test)

## 3. create input pipeline for generating training/testing batches
This code creates an object that you can iterate over to get image-label pairs. One is for the training set, the other is for the test set. You can just run this block.

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 4. Define variables for physical layer
First, we will create complex-valued trainable illumination wave. As described in class, this input illumination field can be represented by a complex-valued 2D matrix, which here will have 28x28 complex-valued entries that interact with each sample of a similar size. In practice, this input field might be obtained by using a spatial light modulator (SLM), an optical element that can be programmed to display an arbitrary phase and/or amplitude pattern pixel by pixel. We will use a phase-only SLM, so that the variable to optimize is a 28x28 array of phases (from 0 to 2pi).

In [0]:
# create the tf variable corresponding to the input illumination phase; initialize to a constant phase:
# (remember this is a weight variable that you will optimize!)
input_illumination_phase = None


Next, we create a complex-valued trainable aperture function. In HW4, we modeled the aperture plane of the microscope using a circular function that was 1 inside the circle (100% transparent) and 0 outside the circle (0% transparent). Please do the same here. For now, in whatever coordinate system you have established, please try to ensure that the radius extends across 8 pixels of the 28 pixels that will define the k-space matrix for this MNIST dataset along one dimension. We may come back later to adjust this radius.

In addition, for extra flexibility, let's add an SLM in the aperture plane. Assume the SLM is a phase-only SLM (only values from 0 to 2pi are allowed).

In [0]:
# the aperture function consists of two parts: 1) the circular aperture as in HW4, and 2) a trainable 28x28 phase array
# create a circular aperture as you did in HW4:
circ_aper = None

# create the variable corresponding to the aperture phase; initialize to a constant phase:
# (remember this is a weight variable that you will optimize!)
aperture_phase = None

## 5. Generate prediction from the physical layer.
Write the physical layer as a function that will be used later. This will follow closely to what you did in HW4, except now all operations must be done with tensorflow operations, for example `tf.signal.fft2d`, `tf.signal.ifft2d`, and `tf.exp`. Note that while often numpy will cast between different data types, tensorflow in general will not, so you will have to use `tf.cast` to go between `tf.float32` and `tf.complex64`. Follow the comments below, which tell you what statements you have to write.

Hint: we recommend subtracting out 1 from simulated noisy images, as we found that it promotes convergence speed. This is possibly because the magnitudes of our simulated phase-only images are close to 1, so the -1 centers the images.

In [0]:
def physical_layer(x_batch):
  # x_batch is a batch of phase-only objects that we created earlier;
  # propagate the field emerging from these objects to the Fourier plane, apply
  # your aperture mask (circular aperture and phase), then propagate it to the
  # image plane;

  # add some gaussian noise with stdev of 0.2, to simulate detector noise:
  noise_sig = .2
  image += None

  # if you didn't already, add color channel singleton dimension in preparation
  # for processing through a CNN:
  image = image[..., tf.newaxis]

  return image-1

## 6. process the simulated image through a CNN
Use your favorite CNN architecture that classifies MNIST or come up with a new one. You may copy a network architecture from a previous TA or class session.

In [0]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    # ...
  def call(self, x):
    # ...

model = CNN()

In [0]:
# the full forward model, which includes both the physical layer and the CNN model:
def forward_model(x_batch):
  p_layer_out = physical_layer(x_batch)
  return model(p_layer_out)

## 7. train!

Let's first look at a few simulated noisy microscope images before training. If your code is correct, the digits should be essentially unrecognizable.

In [0]:
def plot_examples(batch):
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.imshow(batch[i, :, :, 0])
        plt.colorbar()
        plt.axis('off')
    plt.show()

plot_examples(physical_layer(x_train[:25]).numpy())

Write your train loop here, using `tf.GradientTape()` to track the gradients. The variables you have to update are the CNN variables and the two phase matrices that we defined above. Feel free to monitor loss and/or aperture/illumination phases during training. Pick a value for the number of iterations and keep it fixed (run for at least one epoch). Don't forget to monitor test accuracy.

You may find this link useful https://www.tensorflow.org/tutorials/quickstart/advanced.

In [0]:
# train loop
# ...

Let's first look at a few simulated noisy microscope images AFTER training. Do the images look more recognizable?

In [0]:
# plot examples 
plot_examples(physical_layer(x_train[:25]).numpy())

## Questions

Ok, now you've run a machine learning model with a physical layer! You did it! To complete the remainder of the homework, please follow the instructions for each question below, and then respond accordingly with the requested information. If it is helpful, please also copy and paste any major code changes into the question response, to help us assess what you did. Note that this is a pretty open-ended analysis, and everyone may achieve different results depending on the selected parameters -- as long as you get question 1 to work, it's okay if the other questions give uninteresting results!

1. First, using the default hyperparameters based on the instructions and code that we provided, and please run the training under the following conditions:

    a. Allow for the optimization of the complex-valued aperture mask (i.e., the 'aperture phase' variable) and the complex-valued  illumination field that illuminates the sample.

    b. Allow for the optimization of neither the complex-valued aperture mask (i.e., the 'aperture phase' variable) nor the illumination field. Note that this is a control experiment, because only the CNN is trained, and not the physical layers.

    c. Allow for the optimization of only the complex-valued aperture mask (i.e., the 'aperture phase' variable).

    d. Allowing optimization of only the illumination field that illuminates the sample.
    
    For these 4 conditions, report the final test classification accuracies as well as the optimized aperture phase and/or illumination phase. Provide a brief analysis of the results (hint: if you don't see a difference between at least two of the above conditions, something probably went wrong!).
   
2. If you got question 1 to work, good work! Next, let's try changing some of the hyperparameters above. For each of the following questions, rerun the analysis from question 1 under conditions of 1(a) and 1(b).

    a. Try decreasing the amount of noise added at the last step of the physical layer, for example to half and a quarter of the initial value. Try at least two values. Comment on the gap in performance between the physically-optimized and non-physically-optimized cases.

    b. For question 1, you initialized with a constant phase. Next, try to initialize the optimization with random-valued aperture phase and illumination phase. Please report the resulting accuracy and optimized aperture phase and illumination phase.
    
    c. Try changing the diameter of the aperture to two other values. For example, half and double the original diameter used above. Please report the resulting accuracy both with and without the physical layer. 